# Chapter 2
# Go with the Flow: Up and Running with TensorFlow

---

## Hello World

In [1]:
import tensorflow as tf
print(tf.__version__)

1.15.0-rc3


In [4]:
hello = tf.constant("Hello")
world = tf.constant("World")
hello_world = hello + world

with tf.Session() as sess:
    answer = sess.run(hello_world)

print(hello_world)
print(answer)

Tensor("add_2:0", shape=(), dtype=string)
b'HelloWorld'


- The key idea behind computation graphs in TensorFlow is that we first define what computations should take place, and then trigger the computations in an external mechanisms.
- The Session object acts as an interface to the external TensorFlow computation mechanism, and allow us to run parts of the computation graph we have already defined.

---

## MNIST and Softmax Regression

- Put simply, the softmax regression model will figure out, for each pixel in the image,
which digits tend to have high (or low) values in that location.
- All this means is that we sum up the pixel values, each multiplied by a weight, which
we think of as the importance of this pixel in the overall evidence for the digit zero
being in the image.

In [9]:
# import necessary modules
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# some constants
DATA_DIR = '/tmp/data'
NUM_STEPS = 1000
MINIBATCH_SIZE = 100

# read the data
data = input_data.read_data_sets(DATA_DIR, one_hot=True)

# a variable is an element manipulated by the computation, 
# while a placeholder has to be supplied when triggering it
# The size [None, 784] means that each image is of size 784 
# (28×28 pixels unrolled into a single vector), 
# and None is an indicator 
# that we are not currently specifying how many of
# these images we will use at once
x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784, 10]))

y_true = tf.placeholder(tf.float32, [None, 10])
y_pred = tf.matmul(x, W)

# loss function
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_pred, labels=y_true))

# optimization
gd_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

correct_mask = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true, 1))
accuracy = tf.reduce_mean(tf.cast(correct_mask, tf.float32))

with tf.Session() as sess:

    # train
    sess.run(tf.global_variables_initializer())
    for _ in range(NUM_STEPS):
        batch_xs, batch_ys = data.train.next_batch(MINIBATCH_SIZE)
        sess.run(gd_step, feed_dict={x: batch_xs, y_true: batch_ys})

    # test
    ans = sess.run(accuracy, feed_dict={x: data.test.images, y_true: data.test.labels})

print("Accuracy: {:.5}%".format(ans*100))

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Accuracy: 91.66%
